# 演習9.1

バージョン確認

In [1]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

C:\Users\araki\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


b'Hello, TensorFlow!'


In [2]:
import keras
keras.__version__

Using TensorFlow backend.


'2.1.4'

MNISTデータのロード

In [3]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

入力をフラットな配列に変換し、学習データは10,000個、テストデータは1000個だけ利用します。また、元データでは濃淡が0から255までで表現されており、最大値が大きすぎるので、特徴の値の最大値を1としておきます。

In [4]:
# input image dimensions
img_rows, img_cols = 28, 28

X_train = X_train.reshape(X_train.shape[0], img_rows * img_cols)[0:10000]
X_test = X_test.reshape(X_test.shape[0], img_rows * img_cols)[0:1000]

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

出力をone-hotに変換します。

In [5]:
from keras.utils import np_utils
Y_train = np_utils.to_categorical(y_train)[0:10000]
Y_test = np_utils.to_categorical(y_test)[0:1000]

## 多層パーセプトロン (MLP)

準備

In [6]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

まず、中間層1層のモデルを設定します。ユニット数は入力の次元数を考えて、適切に設定してください。活性化関数は、パラメータ[activation](https://keras.io/ja/activations/)で指定します。

In [7]:
input_shape = X_train[0].shape
n_hidden = 200
n_out = len(Y_train[0])  # 10

model = Sequential()
model.add(Dense(n_hidden, input_shape=input_shape, activation='sigmoid'))
model.add(Dense(n_out, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 200)               157000    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2010      
Total params: 159,010
Trainable params: 159,010
Non-trainable params: 0
_________________________________________________________________


損失関数 ([loss](https://keras.io/ja/losses/))、最適化アルゴリズム([optimizer](https://keras.io/ja/optimizers/))、評価基準([metrics](https://keras.io/ja/metrics/))を指定して、モデルをcompileします。

In [8]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(lr=0.1),
              metrics = ['accuracy'])

[fit](https://keras.io/ja/models/sequential/#fit)メソッドで学習します。繰り返し回数はパラメータepocksで、バッチサイズはパラメータbatch\_sizeで指定します。

In [9]:
model.fit(X_train, Y_train, epochs=10, batch_size=200)

Epoch 1/10
10000/10000 [==============================] - 1s 64us/step - loss: 1.9971 - acc: 0.4735
Epoch 2/10
10000/10000 [==============================] - 0s 37us/step - loss: 1.4417 - acc: 0.7194
Epoch 3/10
10000/10000 [==============================] - 0s 37us/step - loss: 1.0801 - acc: 0.7894
Epoch 4/10
10000/10000 [==============================] - 0s 39us/step - loss: 0.8662 - acc: 0.8199
Epoch 5/10
10000/10000 [==============================] - 0s 38us/step - loss: 0.7353 - acc: 0.8394
Epoch 6/10
10000/10000 [==============================] - 0s 37us/step - loss: 0.6488 - acc: 0.8555
Epoch 7/10
10000/10000 [==============================] - 0s 37us/step - loss: 0.5879 - acc: 0.8631
Epoch 8/10
10000/10000 [==============================] - 1s 50us/step - loss: 0.5428 - acc: 0.8703
Epoch 9/10
10000/10000 [==============================] - 0s 38us/step - loss: 0.5076 - acc: 0.8781
Epoch 10/10
10000/10000 [==============================] - 0s 38us/step - loss: 0.4798 - acc: 0.8818

[evaluate](https://keras.io/ja/models/sequential/#evaluate)メソッドで評価します。

In [10]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.5343050398826599
Test accuracy: 0.857


### 中間層の層数を増やす

中間層の層数を増やしてみます。その際、活性化関数sigmoid, tanh, reluのそれぞれの違いを確認します。

In [11]:
model = Sequential()
af = 'sigmoid'
model.add(Dense(n_hidden, input_shape=input_shape, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_out, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 200)               157000    
_________________________________________________________________
dense_4 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_5 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_6 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_7 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                2010      
Total params: 319,810
Trainable params: 319,810
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(lr=0.1),
              metrics = ['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=200)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
10000/10000 [==============================] - 1s 111us/step - loss: 2.3170 - acc: 0.1065
Epoch 2/10
10000/10000 [==============================] - 1s 74us/step - loss: 2.3082 - acc: 0.1111
Epoch 3/10
10000/10000 [==============================] - 1s 68us/step - loss: 2.3109 - acc: 0.1004
Epoch 4/10
10000/10000 [==============================] - 1s 70us/step - loss: 2.3096 - acc: 0.1069
Epoch 5/10
10000/10000 [==============================] - 1s 84us/step - loss: 2.3093 - acc: 0.1043
Epoch 6/10
10000/10000 [==============================] - 1s 78us/step - loss: 2.3092 - acc: 0.1090
Epoch 7/10
10000/10000 [==============================] - 1s 84us/step - loss: 2.3092 - acc: 0.1058
Epoch 8/10
10000/10000 [==============================] - 2s 184us/step - loss: 2.3094 - acc: 0.1005
Epoch 9/10
10000/10000 [==============================] - 3s 299us/step - loss: 2.3095 - acc: 0.1047
Epoch 10/10
10000/10000 [==============================] - 3s 307us/step - loss: 2.3099 - acc: 0.

In [13]:
model = Sequential()
af = 'tanh'
model.add(Dense(n_hidden, input_shape=input_shape, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_out, activation='softmax'))

In [14]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(lr=0.1),
              metrics = ['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=200)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
10000/10000 [==============================] - 1s 105us/step - loss: 0.8084 - acc: 0.7842
Epoch 2/10
10000/10000 [==============================] - 1s 66us/step - loss: 0.3737 - acc: 0.8940
Epoch 3/10
10000/10000 [==============================] - 1s 69us/step - loss: 0.3142 - acc: 0.9093
Epoch 4/10
10000/10000 [==============================] - 1s 82us/step - loss: 0.2830 - acc: 0.9169
Epoch 5/10
10000/10000 [==============================] - 1s 76us/step - loss: 0.2591 - acc: 0.9214
Epoch 6/10
10000/10000 [==============================] - 1s 69us/step - loss: 0.2288 - acc: 0.9340
Epoch 7/10
10000/10000 [==============================] - 1s 70us/step - loss: 0.2148 - acc: 0.9378
Epoch 8/10
10000/10000 [==============================] - 1s 77us/step - loss: 0.1991 - acc: 0.9414
Epoch 9/10
10000/10000 [==============================] - 1s 83us/step - loss: 0.1834 - acc: 0.9458
Epoch 10/10
10000/10000 [==============================] - 1s 79us/step - loss: 0.1706 - acc: 0.949

In [15]:
model = Sequential()
af = 'relu'
model.add(Dense(n_hidden, input_shape=input_shape, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_out, activation='softmax'))

In [16]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(lr=0.1),
              metrics = ['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=200)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
10000/10000 [==============================] - 1s 102us/step - loss: 1.6664 - acc: 0.4743
Epoch 2/10
10000/10000 [==============================] - 1s 74us/step - loss: 0.6938 - acc: 0.7804
Epoch 3/10
10000/10000 [==============================] - 1s 80us/step - loss: 0.3888 - acc: 0.8809
Epoch 4/10
10000/10000 [==============================] - 1s 74us/step - loss: 0.2934 - acc: 0.9142
Epoch 5/10
10000/10000 [==============================] - 1s 91us/step - loss: 0.2629 - acc: 0.9228
Epoch 6/10
10000/10000 [==============================] - 1s 74us/step - loss: 0.2063 - acc: 0.9390
Epoch 7/10
10000/10000 [==============================] - 1s 87us/step - loss: 0.1784 - acc: 0.9483
Epoch 8/10
10000/10000 [==============================] - 1s 96us/step - loss: 0.1818 - acc: 0.9456
Epoch 9/10
10000/10000 [==============================] - 1s 72us/step - loss: 0.1385 - acc: 0.9584
Epoch 10/10
10000/10000 [==============================] - 1s 73us/step - loss: 0.1153 - acc: 0.966

dropoutを入れてみます。学習データとテストデータの正解率の差に注目してください。

In [17]:
from keras.layers.core import Dropout

model = Sequential()
af = 'relu'
model.add(Dense(n_hidden, input_shape=input_shape, activation=af))
model.add(Dropout(0.5))
model.add(Dense(n_hidden, activation=af))
model.add(Dropout(0.5))
model.add(Dense(n_hidden, activation=af))
model.add(Dropout(0.5))
model.add(Dense(n_hidden, activation=af))
model.add(Dropout(0.5))
model.add(Dense(n_hidden, activation=af))
model.add(Dropout(0.5))
model.add(Dense(n_out, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 200)               157000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 200)               40200     
__________

In [18]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(lr=0.1),
              metrics = ['accuracy'])
model.fit(X_train, Y_train, epochs=20, batch_size=200)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/20
10000/10000 [==============================] - 2s 165us/step - loss: 2.3181 - acc: 0.1208
Epoch 2/20
10000/10000 [==============================] - 1s 109us/step - loss: 2.1952 - acc: 0.2043
Epoch 3/20
10000/10000 [==============================] - 1s 121us/step - loss: 2.0431 - acc: 0.2867
Epoch 4/20
10000/10000 [==============================] - 1s 123us/step - loss: 1.8182 - acc: 0.3687
Epoch 5/20
10000/10000 [==============================] - 1s 116us/step - loss: 1.5580 - acc: 0.4446
Epoch 6/20
10000/10000 [==============================] - 1s 109us/step - loss: 1.3601 - acc: 0.5010
Epoch 7/20
10000/10000 [==============================] - 1s 106us/step - loss: 1.2145 - acc: 0.5537
Epoch 8/20
10000/10000 [==============================] - 1s 140us/step - loss: 1.1010 - acc: 0.5978
Epoch 9/20
10000/10000 [==============================] - 1s 115us/step - loss: 0.9937 - acc: 0.6488
Epoch 10/20
10000/10000 [==============================] - 1s 105us/step - loss: 0.9188 - a